In [9]:
# Importiamo le librerie necessarie
from idaes.core import FlowsheetBlock
from pyomo.environ import ConcreteModel, SolverFactory, Var, Constraint, Param, NonNegativeReals
from pyomo.environ import Objective, minimize  # Importiamo anche minimize per l'obiettivo

# Creiamo il modello IDAES
model = ConcreteModel()

# Creiamo un flusso di processo (flowsheet) nel modello senza il parametro default
model.fs = FlowsheetBlock()

# Definiamo i parametri del sistema (esempio di capacità termica e flusso di massa)
model.fs.heat_capacity = Param(initialize=4.18)  # capacità termica in kJ/(kg*K)
model.fs.mass_flow_rate = Param(initialize=1.0)   # portata di massa in kg/s

# Definiamo variabili per la temperatura di ingresso e uscita
model.fs.inlet_temperature = Var(within=NonNegativeReals, initialize=300)  # temperatura ingresso (K)
model.fs.outlet_temperature = Var(within=NonNegativeReals, initialize=350)  # temperatura uscita (K)

# Equazione di bilancio energetico per lo scambiatore di calore
def energy_balance_eqn(model):
    return (model.fs.heat_capacity * model.fs.mass_flow_rate *
            (model.fs.outlet_temperature - model.fs.inlet_temperature)) == 0


2025-03-17 23:08:05 [WARNING] idaes.core.base.flowsheet_model: fs is a top level flowsheet, but dynamic flag set to useDefault. Dynamic flag set to False by default


In [11]:

model.energy_balance = Constraint(rule=energy_balance_eqn)

# Definiamo un obiettivo per minimizzare il consumo energetico
model.obj = Objective(expr=model.fs.heat_capacity * model.fs.mass_flow_rate *
                      (model.fs.outlet_temperature - model.fs.inlet_temperature), sense=minimize)

# Risolviamo il modello
solver = SolverFactory('ipopt')  # Puoi usare 'glpk' o 'ipopt' come solver
solver.solve(model, tee=True)

# Visualizziamo i risultati
print("Temperatura di ingresso:", model.fs.inlet_temperature())
print("Temperatura di uscita:", model.fs.outlet_temperature())


Ipopt 3.13.2: 

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt

This version of Ipopt was compiled from source code available at
    https://github.com/IDAES/Ipopt as part of the Institute for the Design of
    Advanced Energy Systems Process Systems Engineering Framework (IDAES PSE
    Framework) Copyright (c) 2018-2019. See https://github.com/IDAES/idaes-pse.

This version of Ipopt was compiled using HSL, a collection of Fortran codes
    for large-scale scientific computation.  All technical papers, sales and
    publicity material resulting from use of the HSL codes within IPOPT must
    contain the following acknowledgement:
        HSL, a collection of Fortran codes for large-scale scientific
        computation. See http://